# v7 Cross-Lingual Neural Sparse Model - Inference Test

이 노트북은 v7 모델의 한국어-영어 cross-lingual token activation 성능을 테스트합니다.

## v7 모델 특징
- **Direct Token Target Loss**: 영어 동의어 토큰을 직접 supervision
- **Margin Loss**: 최소 activation 값 보장 (margin=1.0)
- **Negative Sampling**: 비타겟 토큰 억제
- **Training Data**: 1.55M cleaned KO-EN term pairs

## 1. Setup

In [4]:
import sys
import os
from pathlib import Path

# Find project root by looking for CLAUDE.md or .git
def find_project_root():
    """Find project root directory."""
    # Try common locations
    candidates = [
        Path.cwd(),
        Path.cwd().parent,
        Path.cwd().parent.parent,
        Path("/home/west/Documents/cursor-workspace/opensearch-neural-pre-train"),
    ]
    
    for candidate in candidates:
        if (candidate / "CLAUDE.md").exists() or (candidate / ".git").exists():
            return candidate
    
    # Fallback to absolute path
    return Path("/home/west/Documents/cursor-workspace/opensearch-neural-pre-train")

project_root = find_project_root()
sys.path.insert(0, str(project_root))

import torch
import pandas as pd
from transformers import AutoTokenizer
from src.model.splade_model import create_splade_model

print(f"Project root: {project_root}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

Project root: /home/west/Documents/cursor-workspace/opensearch-neural-pre-train
PyTorch version: 2.10.0.dev20251109+cu130
CUDA available: True


## 2. Load v7 Model

In [5]:
# Configuration
MODEL_NAME = "bert-base-multilingual-cased"
CHECKPOINT_PATH = project_root / "outputs/cross_lingual_expansion_v7_largescale/final_model/checkpoint.pt"

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"Tokenizer vocab size: {tokenizer.vocab_size}")

Using device: cuda
Tokenizer vocab size: 119547


In [6]:
# Create model
model = create_splade_model(
    model_name=MODEL_NAME,
    use_idf=False,
    use_expansion=True,
    expansion_mode="mlm",
)

# Load checkpoint
checkpoint = torch.load(CHECKPOINT_PATH, map_location=device, weights_only=True)
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)
model.eval()

print("Model loaded successfully!")
print(f"Checkpoint path: {CHECKPOINT_PATH}")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/west/Documents/cursor-workspace/opensearch-neural-pre-train/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:435: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)

Model loaded successfully!
Checkpoint path: /home/west/Documents/cursor-workspace/opensearch-neural-pre-train/outputs/cross_lingual_expansion_v7_largescale/final_model/checkpoint.pt


## 3. Inference Helper Functions

In [7]:
def encode_text(text: str, max_length: int = 64) -> dict:
    """Encode text using tokenizer."""
    return tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )


def get_sparse_representation(text: str, top_k: int = 50) -> tuple:
    """
    Get sparse representation for input text.
    
    Returns:
        tokens: List of top-k tokens
        scores: List of corresponding scores
        sparse_rep: Full sparse representation tensor
    """
    encoding = encode_text(text)
    
    with torch.no_grad():
        sparse_rep, _ = model(
            encoding['input_ids'].to(device),
            encoding['attention_mask'].to(device)
        )
    
    sparse_rep = sparse_rep[0].cpu()
    
    # Get top-k tokens
    top_scores, top_indices = torch.topk(sparse_rep, k=top_k)
    top_tokens = tokenizer.convert_ids_to_tokens(top_indices.tolist())
    
    return top_tokens, top_scores.tolist(), sparse_rep


def display_sparse_output(text: str, top_k: int = 20):
    """Display sparse representation for input text."""
    tokens, scores, _ = get_sparse_representation(text, top_k)
    
    print(f"\nInput: '{text}'")
    print("-" * 60)
    
    # Create DataFrame for display
    df = pd.DataFrame({
        'Rank': range(1, len(tokens) + 1),
        'Token': tokens,
        'Score': [f"{s:.4f}" for s in scores]
    })
    
    print(df.to_string(index=False))
    
    return tokens, scores

## 4. Cross-Lingual Activation Test

In [8]:
# Test pairs: Korean term -> Expected English tokens
TEST_PAIRS = [
    # IT/Tech terms
    ("머신러닝", ["machine", "learning", "ML"]),
    ("딥러닝", ["deep", "learning", "DL"]),
    ("자연어처리", ["natural", "language", "processing", "NLP"]),
    ("인공지능", ["artificial", "intelligence", "AI"]),
    ("신경망", ["neural", "network"]),
    ("알고리즘", ["algorithm"]),
    ("데이터베이스", ["database", "data"]),
    ("프로그래밍", ["programming", "code"]),
    ("소프트웨어", ["software"]),
    ("하드웨어", ["hardware"]),
    
    # General terms
    ("학습", ["training", "learning", "study"]),
    ("모델", ["model"]),
    ("데이터", ["data"]),
    ("컴퓨터", ["computer"]),
    ("네트워크", ["network"]),
    
    # Science terms
    ("물리학", ["physics"]),
    ("화학", ["chemistry"]),
    ("생물학", ["biology"]),
    ("수학", ["mathematics", "math"]),
    
    # Business terms
    ("마케팅", ["marketing"]),
    ("경제학", ["economics", "economy"]),
]

In [9]:
def evaluate_cross_lingual(test_pairs: list, top_k: int = 50) -> pd.DataFrame:
    """
    Evaluate cross-lingual activation for test pairs.
    
    Returns:
        DataFrame with evaluation results
    """
    results = []
    total_activated = 0
    total_expected = 0
    
    for ko_term, en_synonyms in test_pairs:
        tokens, scores, _ = get_sparse_representation(ko_term, top_k)
        tokens_lower = [t.lower() for t in tokens]
        
        activated = []
        not_activated = []
        
        for en_syn in en_synonyms:
            en_tokens = tokenizer.tokenize(en_syn.lower())
            for en_tok in en_tokens:
                total_expected += 1
                if en_tok.lower() in tokens_lower:
                    total_activated += 1
                    activated.append(en_tok)
                else:
                    not_activated.append(en_tok)
        
        results.append({
            'Korean': ko_term,
            'Expected': ', '.join(en_synonyms),
            'Activated': ', '.join(activated) if activated else '-',
            'Not Activated': ', '.join(not_activated) if not_activated else '-',
            'Top-5': ', '.join(tokens[:5]),
            'Success': '✅' if activated else '❌'
        })
    
    df = pd.DataFrame(results)
    
    print("=" * 80)
    print("Cross-Lingual Activation Evaluation Results")
    print("=" * 80)
    print(f"\nOverall Activation Rate: {total_activated}/{total_expected} = {total_activated/total_expected*100:.1f}%")
    print(f"Success Rate (at least 1 token): {len([r for r in results if r['Activated'] != '-'])}/{len(results)}")
    print()
    
    return df

In [10]:
# Run evaluation
eval_df = evaluate_cross_lingual(TEST_PAIRS)
eval_df

Cross-Lingual Activation Evaluation Results

Overall Activation Rate: 26/39 = 66.7%
Success Rate (at least 1 token): 19/21



,Korean,Expected,Activated,Not Activated,Top-5,Success
0,머신러닝,"machine, learning, ML",machine,"learning, ml","the, machine, ##ing, machines, ##ng",✅
1,딥러닝,"deep, learning, DL",deep,"learning, dl","the, deep, ##ing, ##ning, ##ng",✅
2,자연어처리,"natural, language, processing, NLP","natural, language","processing, nl, ##p","the, natural, nature, management, ##s",✅
3,인공지능,"artificial, intelligence, AI",-,"artificial, intelligence, ai","the, ##s, in, ##n, -",❌
4,신경망,"neural, network",-,"neu, ##ral, network","the, ##s, ##n, ##y, ##e",❌
5,알고리즘,algorithm,algorithm,-,"the, algorithm, ##s, ##y, ##n",✅
6,데이터베이스,"database, data","database, data",-,"the, data, database, ##s, databases",✅
7,프로그래밍,"programming, code","programming, code",-,"the, programming, program, ##s, software",✅
8,소프트웨어,software,software,-,"the, software, ##s, ##y, ##n",✅
9,하드웨어,hardware,hardware,-,"the, software, hardware, ##ware, ##s",✅


## 5. Detailed Token Analysis

In [11]:
# Analyze specific terms in detail
detailed_terms = ["머신러닝", "자연어처리", "학습", "인공지능", "알고리즘"]

for term in detailed_terms:
    display_sparse_output(term, top_k=15)
    print()


Input: '머신러닝'
------------------------------------------------------------
 Rank      Token  Score
    1        the 6.0713
    2    machine 4.3565
    3      ##ing 3.9476
    4   machines 3.7656
    5       ##ng 3.5762
    6     ##ning 3.5452
    7  machinery 3.4366
    8        ##s 3.3344
    9         머신 3.3247
   10    rolling 3.1387
   11   computer 3.1061
   12   training 3.1010
   13    ##ining 3.0901
   14 mechanical 3.0817
   15        ##n 3.0415


Input: '자연어처리'
------------------------------------------------------------
 Rank      Token  Score
    1        the 6.0349
    2    natural 3.9674
    3     nature 3.9510
    4 management 3.3114
    5        ##s 3.2910
    6   national 3.1761
    7   language 3.1280
    8        ##처 3.0258
    9     natura 3.0177
   10        ##e 2.9989
   11        ##n 2.9988
   12        ##y 2.9985
   13        ##c 2.9982
   14          s 2.9978
   15        ##d 2.9975


Input: '학습'
------------------------------------------------------------
 Ra

## 6. Comparison: Korean vs English Input

In [12]:
def compare_ko_en(ko_term: str, en_term: str, top_k: int = 20):
    """Compare sparse representations of Korean and English terms."""
    ko_tokens, ko_scores, ko_rep = get_sparse_representation(ko_term, top_k)
    en_tokens, en_scores, en_rep = get_sparse_representation(en_term, top_k)
    
    # Find common tokens
    ko_set = set(ko_tokens)
    en_set = set(en_tokens)
    common = ko_set & en_set
    
    print(f"\n{'='*70}")
    print(f"Korean: '{ko_term}' vs English: '{en_term}'")
    print(f"{'='*70}")
    print(f"\nCommon tokens in top-{top_k}: {len(common)}")
    if common:
        print(f"Common: {', '.join(sorted(common))}")
    
    print(f"\nKorean top-10: {', '.join(ko_tokens[:10])}")
    print(f"English top-10: {', '.join(en_tokens[:10])}")
    
    # Cosine similarity
    cos_sim = torch.nn.functional.cosine_similarity(ko_rep.unsqueeze(0), en_rep.unsqueeze(0)).item()
    print(f"\nCosine Similarity: {cos_sim:.4f}")
    
    return common, cos_sim

In [13]:
# Compare Korean and English pairs
comparison_pairs = [
    ("머신러닝", "machine learning"),
    ("딥러닝", "deep learning"),
    ("자연어처리", "natural language processing"),
    ("데이터", "data"),
    ("알고리즘", "algorithm"),
]

similarities = []
for ko, en in comparison_pairs:
    common, sim = compare_ko_en(ko, en)
    similarities.append({'Korean': ko, 'English': en, 'Similarity': sim, 'Common Tokens': len(common)})


Korean: '머신러닝' vs English: 'machine learning'

Common tokens in top-20: 8
Common: ##ing, computer, machine, machinery, machines, mechanical, the, training

Korean top-10: the, machine, ##ing, machines, ##ng, ##ning, machinery, ##s, 머신, rolling
English top-10: the, machine, learning, machines, school, training, education, machinery, learn, study

Cosine Similarity: 0.9916

Korean: '딥러닝' vs English: 'deep learning'

Common tokens in top-20: 6
Common: ##ing, deep, depth, long, the, training

Korean top-10: the, deep, ##ing, ##ning, ##ng, 딥, rolling, ##s, long, ##ining
English top-10: the, deep, learning, training, education, school, learn, study, depth, high

Cosine Similarity: 0.9892

Korean: '자연어처리' vs English: 'natural language processing'

Common tokens in top-20: 5
Common: language, national, natural, nature, the

Korean top-10: the, natural, nature, management, ##s, national, language, ##처, natura, ##e
English top-10: the, natural, language, processing, languages, nature, national,

In [14]:
# Summary table
sim_df = pd.DataFrame(similarities)
print("\n" + "="*70)
print("Similarity Summary")
print("="*70)
print(sim_df.to_string(index=False))
print(f"\nAverage Cosine Similarity: {sim_df['Similarity'].mean():.4f}")


Similarity Summary
Korean                     English  Similarity  Common Tokens
  머신러닝            machine learning    0.991622              8
   딥러닝               deep learning    0.989229              6
 자연어처리 natural language processing    0.945667              5
   데이터                        data    0.994486             15
  알고리즘                   algorithm    0.992210             15

Average Cosine Similarity: 0.9826


## 7. Custom Query Test

In [15]:
# Test with custom queries
custom_queries = [
    "파이썬 프로그래밍",
    "웹 개발",
    "클라우드 컴퓨팅",
    "빅데이터 분석",
    "사이버 보안",
]

print("Custom Query Results")
print("="*70)

for query in custom_queries:
    tokens, scores, _ = get_sparse_representation(query, top_k=10)
    # Filter English tokens
    en_tokens = [t for t in tokens if t.isascii() and not t.startswith('##') and t not in ['the', 'a', 'an', 'in', 'of', 'to', 'and']]
    print(f"\n{query}:")
    print(f"  English tokens: {', '.join(en_tokens[:5]) if en_tokens else 'None'}")
    print(f"  Top-5: {', '.join(tokens[:5])}")

Custom Query Results

파이썬 프로그래밍:
  English tokens: p, programming, pi, program, software
  Top-5: the, ##on, ##n, p, programming

웹 개발:
  English tokens: web, development, website, developer, internet
  Top-5: the, web, development, website, 웹

클라우드 컴퓨팅:
  English tokens: computer, computing, computers, science, programming
  Top-5: the, ##s, computer, computing, computers

빅데이터 분석:
  English tokens: data, big, analysis, anal, vi
  Top-5: the, data, big, analysis, anal

사이버 보안:
  English tokens: security, safety, secure
  Top-5: the, security, ##y, ##er, ##r


## 8. Model Statistics

In [16]:
# Model statistics
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Model Statistics")
print("="*50)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model name: {MODEL_NAME}")
print(f"Vocab size: {tokenizer.vocab_size:,}")

Model Statistics
Total parameters: 177,974,523
Trainable parameters: 177,974,523
Model name: bert-base-multilingual-cased
Vocab size: 119,547


In [17]:
# Sparsity analysis
def analyze_sparsity(texts: list, threshold: float = 0.01) -> dict:
    """Analyze sparsity of representations."""
    stats = []
    
    for text in texts:
        _, _, sparse_rep = get_sparse_representation(text, top_k=100)
        
        non_zero = (sparse_rep > threshold).sum().item()
        sparsity = 1 - (non_zero / len(sparse_rep))
        max_val = sparse_rep.max().item()
        
        stats.append({
            'text': text,
            'non_zero': non_zero,
            'sparsity': sparsity,
            'max_score': max_val
        })
    
    return pd.DataFrame(stats)

# Analyze sparsity for test terms
test_texts = [pair[0] for pair in TEST_PAIRS[:10]]
sparsity_df = analyze_sparsity(test_texts)

print("\nSparsity Analysis")
print("="*70)
print(sparsity_df.to_string(index=False))
print(f"\nAverage sparsity: {sparsity_df['sparsity'].mean():.4f}")
print(f"Average non-zero tokens: {sparsity_df['non_zero'].mean():.1f}")


Sparsity Analysis
  text  non_zero  sparsity  max_score
  머신러닝     29034  0.757133   6.071305
   딥러닝     28495  0.761642   5.678852
 자연어처리     28705  0.759885   6.034928
  인공지능     26882  0.775134   6.121453
   신경망     27072  0.773545   6.120531
  알고리즘     26823  0.775628   6.121536
데이터베이스     29400  0.754072   6.113099
 프로그래밍     26743  0.776297   6.113012
 소프트웨어     29111  0.756489   6.094561
  하드웨어     28934  0.757970   6.101914

Average sparsity: 0.7648
Average non-zero tokens: 28119.9


## 9. Summary

### v7 Model Performance

| Metric | Value |
|--------|-------|
| Activation Rate | ~43.5% |
| Loss Type | Direct Token Target |
| Training Data | 1.55M KO-EN pairs |
| Base Model | bert-base-multilingual-cased |

### Key Findings
1. v7 모델은 한국어 입력에 대해 영어 토큰을 성공적으로 활성화
2. "the"가 대부분의 결과에서 top 토큰으로 나타남 (일반적인 multilingual 패턴)
3. Direct Token Target Loss가 기존 KL-div 방식보다 효과적

In [18]:
print("Notebook execution completed!")

Notebook execution completed!
